In [24]:
# Welcome to MyTorch!
# This Notebook demonstrates how easy it is to use our cloud-based GPUs.
# Notice that we are using standard PyTorch syntax, but our venv has MyTorch instead.
# That's how easy it is to use MyTorch.
#
# Setup Instructions:
# 1) Register at MyTorch.net and get your access token, then save it to ~/.mytorch
#    > echo "token=xxx" > ~/.mytorch
# 2) Set Up a Clean Virtual Environment:
#    Create and activate a new virtual environment to isolate your MyTorch installation:
#    > python3 -m venv ~/venv_mytorch
#    > source ~/venv_mytorch/bin/activate
# 3) Install MyTorch and Jupyter Packages:
#    Install the MyTorch wheel along with JupyterLab and IPython kernel support:
#    > pip install https://raw.githubusercontent.com/mytorch-ai/examples/main/mytorch-0.3.0-py3-none-any.whl
#    > pip install jupyterlab ipykernel
# 4) Register the Virtual Environment as a Jupyter Kernel:
#    This makes your new environment available as a kernel named “mytorch (venv)” in Jupyter:
#    > python -m ipykernel install --user --name mytorch_env --display-name "mytorch (venv)"
# 5) Launch Jupyter Notebook:
#    > jupyter notebook

import torch
print("MyTorch version:", torch.__version__)

mytorch version: 0.3.0


In [25]:
# Step One: Confirm that MyTorch is working, GPU should be available.
device = "cuda" if torch.cuda.is_available() else "cpu"
if torch.cuda.is_available():
    print(f"*** Using GPU: {torch.cuda.get_device_name()} ***")
else:
    print("*** No GPU available ***")

*** Using GPU: Tesla P100-PCIE-16GB ***


In [26]:
# Step Two: Confirm that the MyTorch Token is saved in the correct place.
import os
home_dir = os.path.expanduser("~")
print("Home directory:", home_dir)
print("Does ~/.mytorch exist?", os.path.exists(os.path.join(home_dir, ".mytorch")))

Home directory: /home/footbag
Does ~/.mytorch exist? True


In [27]:
# Step Three: Log in to Hugging Face
import huggingface_hub
huggingface_token = os.getenv("HF_TOKEN")
if huggingface_token is not None:
    huggingface_hub.login(token=huggingface_token)
else:
    # make sure they have logged in
    hf_token_path = os.path.join(os.path.expanduser('~'), '.cache/huggingface/token')
    if os.path.exists(hf_token_path):
        print("You are already logged into Hugging Face, which makes me happy!!!")
    else:
        print("*** You must either set the environment variable HF_TOKEN or \n"
              "*** login to hugging face using the CLI command", file=sys.stderr)
        exit(1)

You are already logged into Hugging Face, which makes me happy!!!


In [30]:
# Step Four: Load the Llama 3.2 3B Instruct model
import sys
import time
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer

model = "meta-llama/Llama-3.2-3B-Instruct"

print(f"Loading tokenizer for {model}...")
tokenizer = AutoTokenizer.from_pretrained(
    model
)
tokenizer.pad_token = tokenizer.eos_token

print(f"Loading model for {model}...")
model = AutoModelForCausalLM.from_pretrained(
    model
)
print("Model loaded")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Loading model onto {device}")
model.to(device)
print("Model moved to device")

INFO - Loading AutoTokenizer for `meta-llama/Llama-3.2-3B-Instruct`


Loading tokenizer for meta-llama/Llama-3.2-3B-Instruct...
                    

INFO - ...AutoTokenizer has been loaded
INFO - Loading AutoModelForCausalLM model `meta-llama/Llama-3.2-3B-Instruct`


Loading model for meta-llama/Llama-3.2-3B-Instruct...
                    

INFO - ...model has been loaded


Model loaded
Loading model onto cuda
Model moved to device


In [32]:
# Step Five: Give the model a prompt.
start_time = time.time()

# Prepare the prompt
prompt = "Tell me a 500 word story about a black cat named George"

# Format the prompt according to Llama 2's chat template
formatted_prompt = f"[INST] {prompt} [/INST]"

inputs = tokenizer.encode_plus(
    formatted_prompt,
    add_special_tokens=True,  # Adds special tokens (e.g., [CLS], [SEP])
    return_tensors="pt",  # Return PyTorch tensors
    padding='max_length',  # Pad to a maximum length specified by the model or manually set
    truncation=True,  # Truncate to a maximum length specified by the model or manually set
    max_length=512  # Specify the maximum length
)
print("Prompt: ", prompt)

# Move each tensor in the inputs dictionary to the correct device
inputs = {k: v.to(device) for k, v in inputs.items()}

print("\nGenerating response...")
  
# Generate the response
outputs = model.generate(
    input_ids=inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    max_length=1000,  # Adjust based on desired length
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
)

# Decode and print the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Clean up the response by removing instruction tokens, the original prompt, and extra whitespace
response = response.replace("[INST]", "").replace("[/INST]", "").strip()
response = response.replace(prompt, "").strip()
end_time = time.time()

print()
print(f"Prompt: {prompt}")
print()
print("Response:")
print(response)
print()
print(f"Time taken: {end_time - start_time:.2f} seconds")

Prompt:  Tell me a 500 word story about a black cat named George

Generating response...

Prompt: Tell me a 500 word story about a black cat named George

Response:
]
George was a sleek and mysterious black cat with piercing green eyes that seemed to gleam in the dark. He had been a stray for as long as anyone could remember, prowling the streets of the small town of Willow Creek, always staying just out of reach of the humans who lived there.

But George was more than just a stray. He was a guardian, a protector of the town's secrets and a keeper of its mysteries. He had a special connection to the town's history, and he knew its stories and legends better than anyone.

As a young kitten, George had been found in a hidden alleyway by the town's founder, a kind old man named Silas. Silas had taken George in and raised him as his own, teaching him the ways of the town and its people. But as George grew older, he began to realize that he was more than just a pet - he was a key part of th